# Quantinuum Nexus Quick Start


<div class="alert alert-block alert-info"><b>NB:</b> Please follow the Quantinuum Nexus Instructions in the README.md before following along with this notebook.</div>

## Welcome to the Quantinuum Nexus Lab! A hosted Jupyter environment with the pre-installed Python software packages you'll need, including:

* [`pytket`](https://tket.quantinuum.com/api-docs/index.html): Quantinuum's Python SDK for TKET quantum programming tools.
* `pytket-nexus`: the client software for submitting jobs to the Quantinuum Nexus platform

### Step 1. Enable access to the Quantinuum simulator and emulator

We provide three systems for you to target when executing quantum circuits as part of the challenge.

* `H1-1LE`: a noiseless simulator of Quantinuum's 20 qubit `H1-1` quantum computer. This system has shorter queuing times, so we recommend using it for most of the challenge.
* `H1-1E`: a noisy 'emulator' of `H1-1`. This system is much closer to the actual `H1-1` machine, and has longer queuing times. We recommend only using this system when you need to execute your circuit on something closer to the real hardware.
* `H1-1SC`: a 'syntax checker' system for optionally testing things out (it won't run any code but can be used for validation of your usage of the API). 


### Step 2: Try out the example code below to compile and simulate your first circuit.

- Copy this file or its contents into the Quantinuum Nexus hub and run the code below to see some example usage.


## Lets compose a quantum circuit using Pytket

Nexus uses [pytket Circuits](https://cqcl.github.io/pytket/manual/manual_circuit.html). Here we'll create a simple one.

In [ ]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter

my_circuit = Circuit(4,4).H(0).CX(0, 1).H(1).CX(1,2).H(3).CX(2,3)
my_circuit.measure_all()

my_circuit.name = "my-first-circuit"

render_circuit_jupyter(my_circuit)

### Create a Nexus Project

A project is a way to compartmentalize your running jobs and data, allowing you to retrieve and share data easily.

In [ ]:
from pytket.extensions.nexus import Nexus
from datetime import datetime

nexus = Nexus()
my_project = nexus.new_project(name=f"Nexus Spring 2024 Workshop Tutorial - {datetime.now()}")

You should be able to see this project if you visit [https://nexus.quantinuum.com/projects/all](https://nexus.quantinuum.com/projects/all)

### Set up your Backend Configuration

In Quantinuum Nexus, we use `BackendConfig`s to specify the details of the quantum system we want to target.


For this example, we'll use ``QuantinuumConfig``, to configure Nexus to target a simulator for Quantinuum's `H1-1LE` device. 


In [ ]:
from pytket.extensions.nexus import QuantinuumConfig

configuration = QuantinuumConfig(device_name="H1-1LE")

## Set up a NexusBackend

The `Backend` object presents an interface to run jobs on Nexus.

In [ ]:
from pytket.extensions.nexus import NexusBackend

# Then we'll create a NexusBackend using our config and the project we created
backend = NexusBackend(configuration, project=my_project)

### Compile the circuit

Nexus offers compilation with the TKET quantum compiler. This is an easy way to:

* Optimize the circuit
* Convert it to run on a specific quantum device, which might not support all quantum gates.

For the challenge you can use pytket directly to compile circuits, or you can get Nexus to do the compilation for you. If you'd like to disable any changes to your circuit other than making sure that the gateset is correct, you can always set `optimisation_level=0`.

In [ ]:
# Compile the circuit in Nexus
compiled_circuit = backend.get_compiled_circuit(my_circuit, optimisation_level=2)

You should now see the `Compile Job` show up when you click on your project on [https://nexus.quantinuum.com/projects/all](https://nexus.quantinuum.com/projects/all).

If you're curious to see what the compiler did, you can click on the job results and see all the passes the compiler performed on your circuit. 

### Submit the circuit for execution

Nexus will take care of submitting the circuit to your chosen device and storing the results when it has executed.

In this case we are executing the circuit on a simulator running in the Nexus cluster so we should get the results back quickly, however many quantum devices have lengthy queues in which case Nexus will track the job and retrieve the results when they are ready.

In [ ]:
# Run the compiled circuit for 10 shots
handle = backend.process_circuit(compiled_circuit, n_shots=10)


# This will give you a ResultHandle
# As with other pytket-backends, a ResultHandle identifies a particular run of the circuit, which you can then use to keep track of the job status.
print(handle)

In [ ]:
# Get an update on the status
backend.circuit_status(handle)

In [ ]:
# If the job is COMPLETED we can then retrieve the result with the handle
result = backend.get_result(handle)


result.get_distribution()

You can also view the job from the project page on the Quantinuum Nexus website and check out the results there.

<div align="center"> &copy; 2024 by Quantinuum. All Rights Reserved. </div>